In [2]:
import skimage
import matplotlib.image as img
from glob import glob
from final_utils import *
from MultiGap_FC import *
from lime import lime_image
from MultiGap_CNN_FC import *
from tensorflow.keras.preprocessing import image
from skimage.segmentation import mark_boundaries

### Creating model 

In [3]:
model1 = model_multigap_cnn_fc( pca_mg = True, pca_cnn = True)
# model2 = model_multigap_fc()
root_path = generate_root_path()

In [4]:
good_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'good', '*'))
bad_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'bad', '*'))
paths_bench = good_imgs_path + bad_imgs_path
labels_bench = np.concatenate((np.ones(len(good_imgs_path)) ,np.zeros(len(bad_imgs_path))))

### Some useful functions 

In [4]:
def calc_acc(labels, predicted):
    return np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)

def predict(x, model):
    try:
        pred = model.predict(x, verbose=0)
    except:
        x = x[None] #changed 02.08 for evaluator visualizing predictions
        pred = model.predict(x, verbose=0)

    return pred

def predict_from_path(model, paths, resize_func=None, size=None, for_all=False, save_results=None, save_to=None, model_CNN=None):
    #always requires list of paths
    predicted = []
    false_pred = []
    
    for path in paths:
        if 'good' in path:
            label = 1
        elif 'bad' in path:
            label = 0

        img = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
        pred_score = predict(img, model)
    
        if np.argmax(pred_score) != label:
            false_pred.append(path)
            
        predicted.append(pred_score)
    
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
    
#     if save_results:
#         np.save(save_to, np.argmax(predicted, axis=-1))
        
    return predicted, false_pred

def read_and_transform_img(url):
    img = Image.open(url)
    img = img_to_array(img)
#     img = img / 255
#     img = resize_max(img, size=(996,996))
    print(img.shape)
    img = np.expand_dims(img, axis=0)

    return img

### Saparating false predicted images

In [ ]:
predicted, false_pred = predict_from_path(model1, paths_bench, resize_func=resize_max, size=(996, 996))

In [ ]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc}')

### Explanation for false predictions with LIME 

In [5]:
explainer = lime_image.LimeImageExplainer()

In [6]:
path = "XAI/gal-angebot-portrait-1.jpg"

images = read_and_transform_img(path)
print(type(images))
preds = model1.predict(images)
prediction = np.argmax(preds)
pct = np.max(preds)
if prediction == 0:
    print('It\'s no aesthethic!')
elif prediction == 1:
    print('It\'s an aesthethic!')
print(preds)

(1079, 894, 3)
<class 'numpy.ndarray'>
image shape (None, 1079, 894, 3)
Resize max (1079, 894, 3)
image shape after resize max (1, 996, 825, 3)
mg feature vector shape (1, 16928)
image shape after resize border (1, 600, 600, 3)
cnn feature vector shape (None, 2560)
mg feature vector shape after pca (1, 8464)
cnn feature vector shape after pca (None, 1280)
feature vector shape after concat (1, 9744)
feature vector shape after fc (1, 2)
print before return
1/1 [==============================] - 26s 26s/step
It's no aesthethic!
[[9.9987686e-01 1.2312659e-04]]


In [ ]:
i = 0 # index of false predicted image which will be explained

images = read_and_transform_img(false_pred[i])
basename = os.path.basename(false_pred[i])
preds = model1.predict(images)
prediction = np.argmax(preds)
pct = np.max(preds)

if prediction == 0:
    print('It\'s no aesthethic!')
elif prediction == 1:
    print('It\'s an aesthethic!')

print(pct)

In [7]:
explanation = explainer.explain_instance(images[0].astype('double'), model1.predict,  
                                     top_labels=2, hide_color=0, num_samples=1000)

temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

img1 = mark_boundaries(temp_1, mask_1)
img2 = mark_boundaries(temp_2, mask_2)

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 486ms/step


ValueError: Found input variables with inconsistent numbers of samples: [1000, 100]